# Convolutional Neural Network

Convolutional neural network adalah salah satu arsitektur NN yang biasanya dipakai untuk melakukan deteksi objek pada gambar. CNN terdiri dari dua layer khusus, yaitu *convolutional layer* dan *pooling layer*.

*Convolutional layer* berfungsi untuk melakukan ekstraksi fitur dengan cara melakukan proses *filtering*. Pada proses filtering ini, filter akan bergeser ke seluruh gambar yang pergeserannya ditentukan oleh dua operasi yakni *stride* dan *padding* sambil menjalankan operasi *dot product*.

![CNN Works](./images/cnn_works.gif)

Terdapat beberapa langkah yang biasanya perlu dilewati untuk membuat arsitektur CNN, diantaranya:

1. Bagian Konvolusi
2. Pooling
3. Flattening
4. Full Connection

## Stride & Padding

*Stride* adalah operasi yang menentukan seberapa banyak pergeseran dilakukan.

*Padding* adalah operasi yang berfungsi menambahkan dimensi pada input sehingga kita dapat memperoleh ukuran output yang sama atau tidak terlalu kecil dari inputnya. 

![Stride Padding Works](./images/stride_padding_1.gif)

Dimensi output dari hasil filtering ini dengan adanya padding adalah sebagai berikut:

$$H_{0} = \frac{H_{i} - F + 2P}{2}$$

Dimana:

- $W_{i}$ adalah lebar dari input
- $H_{i}$ adalah tinggi dari input
- $F$ adalah ukuran *filter*
- $P$ adalah ukuran *padding*

## Pooling

Pooling layer bergungsi untuk mengurangi dimensi dari fitur yang dihasilkan oleh layer convolutional. Hal ini dilakukan dengan dua cara:

- Max Pooling, hanya mengambil nilai tertinggi dari proporsi yang diambil.
- Average Pooling, mengambil rata-rata dari keseluruhan nilai dari proporsi yang diambil

![Max Average Pooling](./images/max-average-pooling.jpeg)

Biasanya pooling dilakukan karena:

- Invariansi translasional
- Mengurangi jumlah parameter model
- Mengurangi overfitting

Namun pooling juga jarang digunakan karena:

- Dimensi data yang terlalu besar dan kompleks sehingga kita lebih fokus pada masalah *underfitting* ketimbang *overfitting*
- Lebih baik menggunakan dropout
- *Downsampling* pada gambar dapat mengakibatkan hilangnya informasi

## Flattening

Operasi flattening berfungsi untuk memecah matriks yang diperoleh dari operasi konvolusi atau pooling menjadi vektor kolom dengan cara melakukan stacking pada matriks tersebut per barisnya.  

![Flattening](./images/flattening.png)

## Full Connection

Pada layer ini terdiri dari koneksi neuron sederhana seperti MLP pada umumnya.

## Implementasi dengan Keras

**Load library**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import matplotlib.pyplot as plt

**Load dataset**

In [ ]:
train_data = pd.read_csv("./dataset/digit_recognizer/train.csv")
test_data = pd.read_csv("./dataset/digit_recognizer/test.csv")

**Inspeksi Data**

In [ ]:
# informasi terkait tipe dan dimensi data training
train_data.info()

In [ ]:
# informasi terkait tipe dan dimensi data test
test_data.info()

In [ ]:
# cek data yang hilang pada data training
train_data.isna().sum().describe()

In [ ]:
# cek data yang hilang pada data test
test_data.isna().sum().describe()

In [ ]:
# buat variabel untuk menyimpan kelas target
y_train = train_data['label']

y_train

In [ ]:
# buang kolom 'label' pada train data
train_data = train_data.drop(columns='label')

In [ ]:
train_data.shape

In [ ]:
# cek distribusi kelas target
y_train.value_counts().plot.barh();

**Preprocessing data**

In [ ]:
# grayscaling
train_data = train_data/255

test_data = test_data/255

In [ ]:
# reshaping
train_data = train_data.values.reshape(-1,28,28,1)

test_data = test_data.values.reshape(-1,28,28,1)

In [ ]:
y_train = pd.get_dummies(y_train).values

y_train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, y_train, test_size = 0.1, random_state=1000)

In [ ]:
plt.imshow(X_train[0][:,:,0]);

**Inisiasi Model**

In [ ]:
# Inisiasi model sequential
model = Sequential()

**Menambah convolution layer**

In [ ]:
model.add(Conv2D(filters=32, kernel_size=3, strides=3, input_shape=(28,28,1), activation='relu'))

*Keterangan*

API Conv2D umumnya berfungsi untuk melakukan filtering pada data berbentuk gambar. Terdapat parameter yang biasanya sering dipakai dan diatur nilainya yaitu:

- ```filters```: mengatur ukuran dari output atau jumlah filter, dalam hal ini kita akan mengubah gambar yang berukuran 64 x 64 menjadi 32 x 32.
- ```kernel_size```: berfungsi untuk mengatur ukuran matriks konvolusi yang digunakan. Jika diisi berupa nilai integer maka matriksnya akan berbentuk matriks persegi, namun jika diisi dengan tuple (m,n) maka matriks konvolusi akan berukuran m x n.
- ```strides```: berfungsi untuk mengatur seberapa besar perpindahan yang dilakukan oleh matriks konvolusi.
- ```input_shape```: berfungsi untuk memberi tahu bentuk dari input, dalam hal ini berupa gambar berukuran 64 x 64 dengan 3 channel warna.
- ```activation```: berfungsi untuk memberi tahu fungsi aktivasi yang dipakai pada layer ini, dalam hal ini kita menggunakan ```relu```.

**Menambah pooling layer**

In [ ]:
model.add(MaxPooling2D(pool_size=(2, 2)))

*Keterangan*

API MaxPooling2D umumnya berfungsi untuk melakukan proses downsampling pada data dengan mengambil nilai maksimum setelah melalui *convolution layer*. Salah satu parameter yang harus diatur nilainya adalah *pool_size*, yaitu mengatur matriks downsampling yang digunakan.

**Melakukan flattening**

In [ ]:
model.add(Flatten())

**Hubungkan dengan Dense**

In [ ]:
model.add(Dense(units=128, activation='sigmoid'))

In [ ]:
model.add(Dense(units=10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Model fitting**

In [ ]:
model.fit(X_train, Y_train, epochs=5)

**Melihat hasil prediksi**

In [ ]:
hasil_prediksi = model.predict(X_val)

In [ ]:
hasil_prediksi

In [ ]:
hasil_prediksi_crips = np.argmax(hasil_prediksi, axis=1)

In [ ]:
y_val_crips = np.argmax(Y_val, axis=1)

In [ ]:
accuracy_score(y_val_crips, hasil_prediksi_crips)

**Submit Prediksi ke Kaggle**

In [ ]:
submission = model.predict(test_data)

In [ ]:
submission = np.argmax(submission, axis=1)

In [ ]:
pd.DataFrame({"ImageId": range(1,28001), "Label": submission}).to_csv("submission.csv", index=False)

**Latihan**

1. Cek apakah dengan menginisiasi nilai weight di awal akan membuat proses belajar jauh lebih cepat?
2. Apakah dengan membuang layer pooling memiliki pengaruh besar pada akurasi atau kecepatan proses belajar?